In [1]:
from langchain.memory import ConversationBufferMemory
from operator import itemgetter
from langchain_core.runnables import RunnableLambda , RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder , ChatPromptTemplate 
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv 
from langchain_core.documents import Document
load_dotenv()



True

In [22]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
import os
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-001",
    temperature = 0
    
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. and simply answer the question. with short sentences"),
    ("user","{input}")
])

chain = prompt | llm



In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.pydantic_v1 import BaseModel, Field , PydanticOutputParser

load_dotenv()

prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Please answer the following question using 10 words: {question}"
)

class ColorSize(BaseModel):
    color: str = Field(description="color name")
    size: str = Field(description="size names")

llm = ChatOpenAI(
    model_name="gpt-4.1-nano-2025-04-14",
    temperature=0,
)
parser = PydanticOutputParser(pydantic_object=ColorSize)

chain = prompt | llm | parser



In [70]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

load_dotenv()
prompt_template = """당신은 의류 색상 표현을 표준화하는 전문가입니다.

주어진 JSON 리스트 형태의 입력 데이터를 각 product_id 별로 순회하며, 각 product_id 내의 'colors' 리스트에 있는 색상 표현을 다음 규칙에 따라 표준화하여 변환해주세요.

입력 데이터:
{input}

규칙:
1. **색상명 추출 및 불필요 정보 제거:** 입력 문자열에서 순수한 색상명만 추출하고, 색상명이 아닌 불필요한 정보(코드, 숫자, 브랜드명, 제품명, 소재/스타일 정보, 기타 수식어 등)는 모두 제거합니다. 특히 다음 패턴에 유의하여 제거합니다:
   - 대괄호 `[]` 안에 있는 내용 (예: '[17]DARK GRAY', '[기모] 블랙')
   - 숫자 또는 코드 접두사 (예: '01블랙')
   - 언더스코어 `_` 뒤에 붙은 정보 (예: '차콜_쭈리', '차콜_기모')
   - 기타 색상명을 수식하는 불필요한 텍스트 (예: '쭈리', '기모' 등)

2. **색상명 표준화 (영문 소문자):** 추출된 순수한 색상명을 영어 소문자로 변환합니다. 한글 색상명인 경우, 해당 영어 소문자 색상명으로 변환합니다.
   - 예시 변환: '화이트' -> 'white', '블랙' -> 'black', '차콜' -> 'charcoal', '연그레이' -> 'light gray', '웜그레이' -> 'warm gray', '스톤' -> 'stone', '네이비' -> 'navy', '베이지' -> 'beige', 'INDIGO' -> 'indigo', 'DARK GRAY' -> 'dark gray', 'LIGHT BLUE' -> 'light blue'
               'BE' -> 'beige', 'BR' -> 'brown', 'NA' -> 'navy'

3. **줄임말 변환:** 색상 표현의 줄임말인 경우 풀어서 변환합니다. (예: 'Lt Gray' -> 'light gray' 등)

4. **변환 불가 항목 처리:** 위 규칙을 적용하기 어렵거나, 어떤 색상인지 명확히 알 수 없는 경우에는 결과 리스트에 문자열 "pass"로 담아주세요.

출력은 제공된 Pydantic 스키마에 맞는 JSON 배열 형식이어야 합니다. 입력의 각 객체는 출력의 해당 product_id 객체로 변환되어야 하며, 'colors' 리스트는 표준화된 'colors_transformed' 리스트가 되어야 합니다. 추가 설명 없이 JSON만 출력하세요.
"""

prompt = PromptTemplate.from_template(prompt_template)
    

class ColorInfoTransformed(BaseModel):
    # 입력의 각 product_id에 대한 결과를 담을 모델
    product_id : str = Field(..., description="product id")
    colors_transformed: list[str] = Field(... , description="list of standardized color names in lowercase english")

class ProductListTransformed(BaseModel):
    # 전체 결과가 이 모델의 리스트가 되도록 정의 (optional, but can help guide LLM)
    products: list[ColorInfoTransformed] = Field(description="list of products with standardized colors")

llm = ChatOpenAI(
    model_name="gpt-4.1-nano-2025-04-14",
    temperature=0,
)

parser = JsonOutputParser(pydantic_object=ProductListTransformed)

chain = prompt | llm  | parser



In [55]:
import json
with open("./musinsa_product_detail_info.json", "r" , encoding="utf-8") as f:
    data = json.load(f)

In [73]:
colors = []
for d in data[:100]:
    product_id = d["product_id"]
    if d.get("color_size_info") != "failed" and len(d.get("color_size_info")) > 1:
        product_d = {
            "product_id" : product_id,
            "colors" : [c for c in d["color_size_info"].keys()]
        }
        colors.append(product_d)

In [75]:
result = chain.invoke({"input" : colors} , force_json=True)

In [79]:
for i in range(len(result)):
    product_id = result[i]["product_id"]
    origin = colors[i]["colors"]
    output = result[i]["colors_transformed"]
    print(product_id , origin , output)


4175458 ['화이트', '차콜', '블랙'] ['white', 'charcoal', 'black']
4979827 ['LRAMCTR702P_화이트', 'LRAMCTR702P_블랙', 'LKTMCTR701M_화이트', 'LKTMCTR701M_라이트그레이', 'LKTMCTR701M_차콜', 'LKTMCTR701M_네이비', 'LKTMCTR701M_블랙', 'LKTMCTR702M_라이트그레이', 'LKTMCTR702M_네이비', 'LKTMCTR702M_차콜', 'LSBMCTR701M_화이트', 'LSBMCTR701M_블랙', 'LSBMCTR702M_화이트', 'LSBMCTR702M_블랙', 'LSBMCTR703M_화이트', 'LSBMCTR703M_블랙', 'LHTMCTR701M_화이트', 'LHTMCTR701M_블랙', 'LHTMCTR702M_화이트', 'LHTMCTR702M_블랙', 'LKTMCTR751M_화이트', 'LKTMCTR751M_블랙', 'LKTMCTR752M_화이트', 'LKTMCTR752M_블랙'] ['white', 'black', 'white', 'light gray', 'charcoal', 'navy', 'black', 'light gray', 'navy', 'charcoal', 'white', 'black', 'white', 'black', 'white', 'black', 'white', 'black', 'white', 'black', 'white', 'black', 'white', 'black']
4748535 ['RED BEAN', 'ALMOND', 'SOBORO', 'NAVY', 'PISTACHIO'] ['red bean', 'almond', 'soboro', 'navy', 'pistachio']
4984372 ['A.린넨 라이크 카라넥 반팔 니트_블랙', 'A.린넨 라이크 카라넥 반팔 니트_아이보리', 'A.린넨 라이크 카라넥 반팔 니트_베이지', 'A.린넨 라이크 카라넥 반팔 니트_그린', 'A.린넨 라이크 카라넥 반팔 니트_네이

In [5]:
from langchain_core.tools import tool
@tool
def add(a:int , b:int):
    '''add a and b and then return a+b'''
    return a+b
d = {"a":10,"b":20}
add.run(d) , add.invoke(d) 

(30, 30)

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}")
    ]
)


In [20]:
memory = ConversationBufferMemory(
    return_messages = True , 
    memory_key = "chat_histroy"
    
)

In [31]:
runnable = RunnablePassthrough.assign(
    messages = RunnableLambda(memory.load_memory_variables) | itemgetter("chat_histroy")
)

In [32]:
runnable.invoke({"test" : "abc"})

{'test': 'abc', 'messages': []}

In [35]:
chain = runnable | prompt
res = chain.invoke({"input" : "안녕"})

In [47]:
res

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕', additional_kwargs={}, response_metadata={})])

In [3]:
import requests
from langchain_community.embeddings import JinaEmbeddings
from numpy import dot
from numpy.linalg import norm
from PIL import Image

from dotenv import load_dotenv

load_dotenv()


multimodal_embeddings = JinaEmbeddings(
    model_name="jina-clip-v1",
)



In [ ]:
texts = ["a dog", "a cat" , "This is a full-body studio shot of a cute Pembroke Welsh Corgi puppy, sitting with one eye winking and its tongue slightly out, against a clean white background. The puppy has a light brown and white coat.", "강아지의 정면 사진"]
images= ["./dog.jpg", "./dog2.jpg"]


# 리스트 반환 List[List[float]]
text_embeddings = multimodal_embeddings.embed_documents(texts)
image_embedding = multimodal_embeddings.embed_images(images)



In [6]:
import numpy as np
# cosine_similarity1 = np.dot(image_embedding[0], text_embeddings[0]) / (
#     np.linalg.norm(image_embedding[0]) * np.linalg.norm(text_embeddings[0])
# )
# cosine_similarity2 = np.dot(image_embedding[0], text_embeddings[1]) / (
#     np.linalg.norm(image_embedding[0]) * np.linalg.norm(text_embeddings[1])
# )

# cosine_similarity3 = np.dot(image_embedding[0], text_embeddings[2]) / (
#     np.linalg.norm(image_embedding[0]) * np.linalg.norm(text_embeddings[2])
# )

# cosine_similarity4 = np.dot(image_embedding[0], text_embeddings[3]) / (
#     np.linalg.norm(image_embedding[0]) * np.linalg.norm(text_embeddings[3])
# )
np.dot(image_embedding[0] ,image_embedding[0])

np.float64(1.0000001798071025)

In [23]:
cosine_similarity1, cosine_similarity2 , cosine_similarity3 , cosine_similarity4 

(np.float64(0.17015653410642279),
 np.float64(0.057199674549987437),
 np.float64(0.3730112902897924),
 np.float64(0.075123804692477))

In [26]:
np.dot(image_embedding[0]  , text_embeddings[0])

np.float64(0.17015653806414158)